In [1]:
import datetime
from datetime import date
from datetime import datetime
import requests
import pprint
from bs4 import BeautifulSoup
import pandas as pd
import re
import pickle
from urllib.request import urlopen
import glob
# import altair as alt
import time
import itertools
import numpy as np
import operator

pd.options.display.max_columns = 1000


In [6]:
#Getting Player Info Files
# Sravan's paths
# player_info_england = pd.read_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/Dataframes/player_data_df_england.csv')
# player_info_italy = pd.read_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/Dataframes/player_data_df_italy.csv')
# player_info_spain = pd.read_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/Dataframes/player_data_df_spain.csv')
# player_info_france = pd.read_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/Dataframes/player_data_df_france.csv')
# player_info_germany = pd.read_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/Dataframes/player_data_df_germany.csv')

# # Renzo's paths
player_info_england = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_england.csv')
player_info_italy = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_italy.csv')
player_info_spain = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_spain.csv')
player_info_france = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_france.csv')
player_info_germany = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_germany.csv')

# Victor's Paths
# player_info_england = pd.read_csv('G:/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_england.csv')
# player_info_italy = pd.read_csv('G:/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_italy.csv')
# player_info_spain = pd.read_csv('G:/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_spain.csv')
# player_info_france = pd.read_csv('G:/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_france.csv')
# player_info_germany = pd.read_csv('G:/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_germany.csv')


In [7]:
# Load TM Profile Datasets

df_1 = pd.read_csv('player_profile_df_1.csv')
df_2 = pd.read_csv('player_profile_df_2.csv')
df_3 = pd.read_csv('player_profile_df_3.csv')

tm_profile_df = pd.concat([df_1, df_2])
tm_profile_df = pd.concat([tm_profile_df, df_3])

In [8]:
player_inf_lst = [player_info_england, player_info_italy, player_info_spain, player_info_france, player_info_germany]
player_info_df = pd.concat(player_inf_lst)
player_info_df.shape

(35827, 15)

In [9]:
# Cleaning repeated players

player_info_df_nodups = player_info_df.drop_duplicates()
print(len(player_info_df))
print(len(player_info_df_nodups))

35827
10720


In [10]:
#Loading Player Injuries

# Sravan's Path
# player_injuries_df = pd.read_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/Dataframes/player_injuries_df.csv')

# Renzo's Path
player_injuries_df = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_injuries_df.csv')

# Victor's Path
# player_injuries_df = pd.read_csv('G:/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_injuries_df.csv')


len(player_injuries_df)

55216

In [11]:
print(player_injuries_df.columns)
print(player_info_df_nodups.columns)

Index(['Season', 'Injury', 'from', 'until', 'Days', 'Games missed', 'TMId',
       'name'],
      dtype='object')
Index(['FBRefId', 'PlayerName', 'PlayerFullName', 'Nationality', 'Photo',
       'Birth', 'Height', 'Weight', 'Position', 'Foot', 'FootAbility',
       'InternationalReputation', 'PlayerUrl', 'Twitter', 'Instagram'],
      dtype='object')


In [12]:
# Load TMID to FBRefID reference table

# Sravan's Path
# fbref_to_tm_df = pd.read_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/CSV files/fbref_to_tm_mapping.csv', encoding='latin-1')

# Renzo's Path
fbref_to_tm_df = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/CSV files/fbref_to_tm_mapping.csv')

# Victor's Path
# fbref_to_tm_df = pd.read_csv('G:/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/CSV files/fbref_to_tm_mapping.csv', encoding='latin-1')


In [13]:
# Stripping FBRefID and TMID from other information and creating individual columns for them

# fbref_to_tm_df['FBRefID'] = fbref_to_tm_df['UrlFBref'].apply(lambda player: player[(player.find("players/") + len("players/")):(player.find("/matchlogs"))])

fbref_to_tm_df['FBRefID'] = fbref_to_tm_df['UrlFBref'].str.split('/').str[5]
fbref_to_tm_df['TMID'] = fbref_to_tm_df['UrlTmarkt'].str.split('/').str[6]

fbref_to_tm_df.head()


,PlayerFBref,UrlFBref,UrlTmarkt,TmPos,FBRefID,TMID
0,Aaron Connolly,https://fbref.com/en/players/27c01749/Aaron-Co...,https://www.transfermarkt.com/aaron-connolly/p...,Centre-Forward,27c01749,434207
1,Aaron Cresswell,https://fbref.com/en/players/4f974391/Aaron-Cr...,https://www.transfermarkt.com/aaron-cresswell/...,Left-Back,4f974391,92571
2,Aarón Escandell,https://fbref.com/en/players/67669ce7/Aaron-Es...,https://www.transfermarkt.com/aaron-escandell/...,Goalkeeper,67669ce7,284430
3,Aaron Herzog,https://fbref.com/en/players/565c3fe4/Aaron-He...,https://www.transfermarkt.com/aaron-herzog/pro...,Attacking Midfield,565c3fe4,276566
4,Aaron Hickey,https://fbref.com/en/players/1780bb4a/Aaron-Hi...,https://www.transfermarkt.com/aaron-hickey/pro...,Left-Back,1780bb4a,591949


In [14]:
# converting TMId to string in player_injuries_df

player_injuries_df['TMId'] = player_injuries_df['TMId'].astype(str)
# player_injuries_df.head()

In [15]:
# Merging on intersection of player_injuries_df and fbref_to_tm_df on columns TMId and TMID respectively

player_injuries_df_2 = pd.merge(left=player_injuries_df, right=fbref_to_tm_df, left_on='TMId', right_on='TMID', how='inner')

# player_injuries_df_2.head()

In [16]:
# Determining number of unique player in our data

len(player_injuries_df_2['name'].unique())

4614

In [17]:
# removing duplicates from player_info_df

player_info_df = player_info_df.drop_duplicates()
player_info_df.head()

,FBRefId,PlayerName,PlayerFullName,Nationality,Photo,Birth,Height,Weight,Position,Foot,FootAbility,InternationalReputation,PlayerUrl,Twitter,Instagram
0,3bb7b8b4,Ederson,Ederson Santana de Moraes,Brazil,https://fbref.com/req/202005121/images/headsho...,"August 17, 1993",188.0,89.0,GK,LEFT,82.0,5x Domestic League Champion,https://fbref.com/en/players/3bb7b8b4/Ederson,https://twitter.com/edersonmoraes93,https://instagram.com/ederson93/
1,31c69ef1,Rúben Dias,Rúben Dias,Portugal,https://fbref.com/req/202005121/images/headsho...,"May 14, 1997",186.0,82.0,DF (CB),RIGHT,84.0,2x Domestic League Champion,https://fbref.com/en/players/31c69ef1/Ruben-Dias,NaN,https://instagram.com/rubendias/
2,6434f10d,Rodri,Rodrigo Hernández Cascante,Spain,https://fbref.com/req/202005121/images/headsho...,"June 22, 1996",190.0,78.0,MF (CM-DM),RIGHT,85.0,2020-21 Premier League Champion,https://fbref.com/en/players/6434f10d/Rodri,NaN,NaN
3,b400bde0,Raheem Sterling,Raheem Shaquille Sterling,Jamaica,https://fbref.com/req/202005121/images/headsho...,"December 8, 1994",170.0,70.0,FW-MF (AM-WM),RIGHT,88.0,3x Premier League Champion||2018-19 FWA Footba...,https://fbref.com/en/players/b400bde0/Raheem-S...,https://twitter.com/sterling7,https://instagram.com/sterling7/
4,bd6351cd,João Cancelo,João Pedro Cavaco Cancelo,Portugal,https://fbref.com/req/202005121/images/headsho...,"May 27, 1994",182.0,66.0,"DF-MF (CB-FB, right)",RIGHT,87.0,2x Domestic League Champion,https://fbref.com/en/players/bd6351cd/Joao-Can...,NaN,https://instagram.com/jcancelo2/


In [19]:
player_birth_height_weight_foot = player_info_df[['FBRefId', 'Height', 'Weight', 'Foot', 'Birth']]

# Sravan's path
# player_birth_height_weight_foot.to_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/Dataframes/player_profile_weight.csv', index=False)

# Renzo's path
player_birth_height_weight_foot.to_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_profile_weight.csv', index=False)

In [20]:
# Merging Player Injuries with FBRef Profiles

player_injuries_info_df = pd.merge(left=player_injuries_df_2, right=player_info_df, left_on='FBRefID', right_on='FBRefId', how='inner')

len(player_injuries_info_df)

32584

In [21]:
# finding number of unique players
 
len(player_injuries_info_df['name'].unique())

4586

In [22]:
# Merge with TM Profile information

tm_profile_df['TMId'] = tm_profile_df['TMId'].astype(str)
player_injuries_profile_final = pd.merge(left=player_injuries_info_df, right=tm_profile_df, left_on='TMId', right_on='TMId', how='inner')
player_injuries_profile_final.head()

,Season,Injury,from,until,Days,Games missed,TMId,name_x,PlayerFBref,UrlFBref,UrlTmarkt,TmPos,FBRefID,TMID,FBRefId,PlayerName,PlayerFullName,Nationality,Photo,Birth,Height,Weight,Position,Foot,FootAbility,InternationalReputation,PlayerUrl,Twitter,Instagram,Place of birth:,Citizenship:,Height:,Position:,Contract expires:,Former International:,Caps/Goals:,League level:,Joined:,Contract until:,name_y,Date of birth,Last club:,Most games for:,Unknown since:,Date of birth:,Age:,Foot:,Player agent:,Current club:,Agent:,National player:,Name in home country:,Outfitter:,Current international:,Social-Media:,Full name:,On loan from:,Contract there expires:,Contract option:,Retired since:,club:,Without Club since:,2nd club:,country:,Company:,Date of death:,Career break since:,3nd club:,Ban since:
0,18/19,Hamstring Injury,"Sep 29, 2018","Oct 27, 2018",28 days,5,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,71672fa0,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,16/17,Calf Strain,"Mar 18, 2017","Apr 17, 2017",30 days,5,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,71672fa0,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15/16,Calf Injury,"Mar 3, 2016","Mar 31, 2016",28 days,5,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,71672fa0,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,14/15,Neck Injury,"May 11, 2015","May 25, 2015",14 days,2,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,71672fa0,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13/14,Shoulder Injury,"Apr 24, 2014","Jul 17, 2014",84 days,4,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermark

In [23]:
# Creating new columns with features

player_injuries_profile_final = player_injuries_profile_final[player_injuries_profile_final['from'] != '-']
player_injuries_profile_final = player_injuries_profile_final[player_injuries_profile_final['until'] != '-']
player_injuries_profile_final['injury_year'] = player_injuries_profile_final['from'].apply(lambda x: datetime.strptime(x, '%b %d, %Y').year)
player_injuries_profile_final['injury_week'] = player_injuries_profile_final['from'].apply(lambda x: datetime.strptime(x, '%b %d, %Y').strftime('%V'))
player_injuries_profile_final['release_week'] = player_injuries_profile_final['until'].apply(lambda x: datetime.strptime(x, '%b %d, %Y').strftime('%V'))
player_injuries_profile_final['from'] = pd.to_datetime(player_injuries_profile_final['from'])
player_injuries_profile_final['until'] = pd.to_datetime(player_injuries_profile_final['until'])

In [24]:
# Exploding Dataframe to weekly basis

# player_injuries_profile_final['from']=player_injuries_profile_final.apply(lambda x : pd.date_range(start=x['from'], end=x['until'], freq='W') ,axis=1)
# player_injuries_profile_final = player_injuries_profile_final.explode('from')
# player_injuries_profile_final['until']=player_injuries_profile_final['from']+pd.tseries.offsets.MonthEnd()

# player_injuries_profile_final.head(50)

def ran_week(row):
        return list(pd.date_range(row['from'], row['until'], freq='w'))

# def ran_year(row):
    # return list(pd.date_range(row['from'], row['until'], freq='y'))

player_injuries_profile_final['injury_week'] = player_injuries_profile_final['injury_week'].astype(int)
player_injuries_profile_final['release_week'] = player_injuries_profile_final['release_week'].astype(int)
player_injuries_profile_final['current_week'] = player_injuries_profile_final.apply(ran_week, axis = 1)
# player_injuries_profile_final['current_year'] = player_injuries_profile_final.apply(ran_year, axis = 1)

In [25]:
player_injuries_profile_final = player_injuries_profile_final.explode('current_week')

In [26]:
player_injuries_profile_final = player_injuries_profile_final.reset_index(drop=True)
player_injuries_profile_final.loc[pd.isna(player_injuries_profile_final['current_week']), 'current_week'] = player_injuries_profile_final['from']

In [27]:
# player_injuries_profile_final[player_injuries_profile_final['name_x'] == 'Petr Cech'].tail(60)
player_injuries_profile_final = player_injuries_profile_final.rename({'current_week':'current_date'}, axis=1)

In [28]:
player_injuries_profile_final['current_year'] = player_injuries_profile_final['current_date'].apply(lambda x: x.year)#strftime('%W'))
player_injuries_profile_final['current_week'] = player_injuries_profile_final['current_date'].apply(lambda x: x.week)

In [29]:
player_injuries_profile_final['current_week'] = player_injuries_profile_final['current_week'].fillna(player_injuries_profile_final['injury_week'])
player_injuries_profile_final['current_year'] = player_injuries_profile_final['current_year'].fillna(player_injuries_profile_final['injury_year'])

In [30]:
player_injuries_profile_final.head()

,Season,Injury,from,until,Days,Games missed,TMId,name_x,PlayerFBref,UrlFBref,UrlTmarkt,TmPos,FBRefID,TMID,FBRefId,PlayerName,PlayerFullName,Nationality,Photo,Birth,Height,Weight,Position,Foot,FootAbility,InternationalReputation,PlayerUrl,Twitter,Instagram,Place of birth:,Citizenship:,Height:,Position:,Contract expires:,Former International:,Caps/Goals:,League level:,Joined:,Contract until:,name_y,Date of birth,Last club:,Most games for:,Unknown since:,Date of birth:,Age:,Foot:,Player agent:,Current club:,Agent:,National player:,Name in home country:,Outfitter:,Current international:,Social-Media:,Full name:,On loan from:,Contract there expires:,Contract option:,Retired since:,club:,Without Club since:,2nd club:,country:,Company:,Date of death:,Career break since:,3nd club:,Ban since:,injury_year,injury_week,release_week,current_date,current_year,current_week
0,18/19,Hamstring Injury,2018-09-29,2018-10-27,28 days,5,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,71672fa0,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,39,43,2018-09-30,2018,39
1,18/19,Hamstring Injury,2018-09-29,2018-10-27,28 days,5,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,71672fa0,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,39,43,2018-10-07,2018,40
2,18/19,Hamstring Injury,2018-09-29,2018-10-27,28 days,5,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,71672fa0,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,39,43,2018-10-14,2018,41
3,18/19,Hamstring Injury,2018-09-29,2018-10-27,28 days,5,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,71672fa0,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,39,43,

In [31]:
# player_injuries_profile_final[player_injuries_profile_final['name_x'] == 'Petr Cech'].tail(60)

In [32]:
# Saving player_injuries_profile_final to Google Drive

# Renzo's path

player_injuries_profile_final.to_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_injuries_profile_final.csv', index=False)

In [33]:
# Loading Match Dataframes

# Renzo's Path
total_match_logs_df = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/consolidated_df.csv')

# Sravan's Path
# total_match_logs_df = pd.read_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/Dataframes/consolidated_df.csv')

# Victor's Path
# total_match_logs_df = pd.read_csv('G:/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/consolidated_df.csv')

/Users/renzomaldonado/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (19,20,21,23,24,25,26,27,28,29,30,31,32,33,34,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [34]:
total_match_logs_df[total_match_logs_df['FBRefID'] == '0000acda']

,Date,Day,Comp,Round,Venue,Result,Squad,Opponent,Start,Pos,Min,Gls,Ast,PK,PKatt,Sh,SoT,CrdY,CrdR,Touches,Press,Tkl,Int,Blocks,xG,npxG,xA,SCA,GCA,Cmp,Att,Cmp%,Prog,Carries,Prog.1,Succ,Att.1,name,FBRefID,Fls,Fld,Off,Crs,TklW,OG,PKwon,PKcon
100761,2015-08-16,Sun,Coppa Italia,Third round,Home,W 4–1,Torino,Pescara,N,"On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play",NaN,NaN,NaN,"On matchday squad, but did not play",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Marco-Benassi,0000acda,"On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play"
100762,2015-08-23,Sun,Serie A,Matchweek 1,Away,W 2–1,Torino,Frosinone,Y,CM,78,0,0,0,0,3,0,0,0,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Marco-Benassi,0000acda,0,0,0,3,1,0,0,0
100763,2015-08-30,Sun,Serie A,Matchweek 2,Home,W 3–1,Torino,Fiorentina,Y,CM,64,0,0,0,0,1,0,0,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Marco-Benassi,0000acda,2,0,0,1,1,0,0,0
100764,2015-09-13,Sun,Serie A,Matchweek 3,Away,D 2–2,Torino,Hellas Verona,N,"On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play",NaN,NaN,NaN,"On matchday squad, but did not play",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Marco-Benassi,0000acda,"On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play"
100765,2015-09-20,Sun,Serie A,Matchweek 4,Home,W 2–0,Torino,Sampdoria,N,NaN,7,0,0,0,0,1,1,0,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Marco-Benassi,0000acda,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2806400,2014-04-13,Sun,Serie A,Matchweek 33,Home,L 2–4,Livorno,Chievo,Y,NaN,90,0,NaN,0,NaN,NaN,NaN,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Marco-Benassi,0000acda,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
2806401,2014-04-19,Sat,Serie A,Matchweek 34,Away,L 0–3,Livorno,Milan,Y,NaN,81,0,NaN,0,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Marco-Benassi,0000acda,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
2806402,2014-04-27,Sun,Serie A,Matchweek 35,Home,L 0–2,Livorno,Lazio,N,"On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play",NaN,NaN,NaN,"On matchday squad, but did not play",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Marco-Benassi,0000acda,"On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not pl

In [35]:
# Creating 'week' and 'year' column in total_match_logs_df

total_match_logs_df['week'] = pd.to_datetime(total_match_logs_df['Date']).dt.week
total_match_logs_df['year'] = pd.to_datetime(total_match_logs_df['Date']).dt.year

<ipython-input-35-c9fe56b8ae89>:3: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  total_match_logs_df['week'] = pd.to_datetime(total_match_logs_df['Date']).dt.week


In [36]:
print(len(total_match_logs_df.columns))
print(len(player_injuries_profile_final.columns))

49
75


In [37]:
# Re-engineer Columns
# Results
total_match_logs_df.loc[total_match_logs_df['Result'].str[0] == 'W', 'Won'] = 1
total_match_logs_df.loc[total_match_logs_df['Result'].str[0] != 'W', 'Won'] = 0

total_match_logs_df.loc[total_match_logs_df['Result'].str[0] == 'L', 'Loss'] = 1
total_match_logs_df.loc[total_match_logs_df['Result'].str[0] != 'L', 'Loss'] = 0

total_match_logs_df.loc[total_match_logs_df['Result'].str[0] == 'D', 'Draw'] = 1
total_match_logs_df.loc[total_match_logs_df['Result'].str[0] != 'D', 'Draw'] = 0

# Starts
total_match_logs_df.loc[total_match_logs_df['Start'] == 'Y', 'Games_Start'] = 1
total_match_logs_df.loc[total_match_logs_df['Start'] != 'Y', 'Games_Start'] = 0

total_match_logs_df.head()

,Date,Day,Comp,Round,Venue,Result,Squad,Opponent,Start,Pos,Min,Gls,Ast,PK,PKatt,Sh,SoT,CrdY,CrdR,Touches,Press,Tkl,Int,Blocks,xG,npxG,xA,SCA,GCA,Cmp,Att,Cmp%,Prog,Carries,Prog.1,Succ,Att.1,name,FBRefID,Fls,Fld,Off,Crs,TklW,OG,PKwon,PKcon,week,year,Won,Loss,Draw,Games_Start
0,2021-08-07,Sat,DFB-Pokal,First round,Away,W 6–3,Arminia,SpVgg Bayreuth,Y,RB,89.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cedric-Brunner,f0a37d36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31,2021,1.0,0.0,0.0,1.0
1,2021-08-14,Sat,Bundesliga,Matchweek 1,Home,D 0–0,Arminia,Freiburg,Y,RB,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,51.0,7.0,1.0,2.0,4.0,0.0,0.0,0.0,1.0,0.0,25.0,40.0,62.5,3.0,17.0,1.0,0.0,1.0,Cedric-Brunner,f0a37d36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32,2021,0.0,0.0,1.0,1.0
2,2021-08-21,Sat,Bundesliga,Matchweek 2,Away,D 1–1,Arminia,Greuther Fürth,Y,RB,66.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,5.0,1.0,0.0,0.0,0.0,0.0,0.2,2.0,1.0,18.0,28.0,64.3,2.0,16.0,3.0,0.0,0.0,Cedric-Brunner,f0a37d36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33,2021,0.0,0.0,1.0,1.0
3,2021-08-28,Sat,Bundesliga,Matchweek 3,Home,D 1–1,Arminia,Eint Frankfurt,Y,RB,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61.0,11.0,1.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,25.0,49.0,51.0,0.0,28.0,3.0,0.0,0.0,Cedric-Brunner,f0a37d36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34,2021,0.0,0.0,1.0,1.0
4,2021-09-12,Sun,Bundesliga,Matchweek 4,Away,L 1–3,Arminia,M'Gladbach,Y,RB,90.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,53.0,14.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,25.0,41.0,61.0,3.0,24.0,0.0,1.0,1.0,Cedric-Brunner,f0a37d36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36,2021,0.0,1.0,0.0,1.0


In [38]:
total_match_logs_df = total_match_logs_df.replace('On matchday squad, but did not play', 0)
total_match_logs_df['Min'] = total_match_logs_df['Min'].astype(float)
total_match_logs_df['Gls'] = total_match_logs_df['Gls'].astype(float)
total_match_logs_df['Ast'] = total_match_logs_df['Ast'].astype(float)
total_match_logs_df['PK'] = total_match_logs_df['PK'].astype(float)
total_match_logs_df['PKatt'] = total_match_logs_df['PKatt'].astype(float)
total_match_logs_df['Sh'] = total_match_logs_df['Sh'].astype(float)
total_match_logs_df['SoT'] = total_match_logs_df['SoT'].astype(float)
total_match_logs_df['Att.1'] = total_match_logs_df['Att.1'].astype(float)
total_match_logs_df['Fls'] = total_match_logs_df['Fls'].astype(float)
total_match_logs_df['Fld'] = total_match_logs_df['Fld'].astype(float)
total_match_logs_df['Off'] = total_match_logs_df['Off'].astype(float)
total_match_logs_df['Crs'] = total_match_logs_df['Crs'].astype(float)
total_match_logs_df['TklW'] = total_match_logs_df['TklW'].astype(float)
total_match_logs_df['OG'] = total_match_logs_df['OG'].astype(float)
total_match_logs_df['PKwon'] = total_match_logs_df['PKwon'].astype(float)
total_match_logs_df['PKcon'] = total_match_logs_df['PKcon'].astype(float)
total_match_logs_df['CrdY'] = total_match_logs_df['CrdY'].astype(float)
total_match_logs_df['CrdR'] = total_match_logs_df['CrdR'].astype(float)
total_match_logs_df['Touches'] = total_match_logs_df['Touches'].astype(float)
total_match_logs_df['Press'] = total_match_logs_df['Press'].astype(float)
total_match_logs_df['Tkl'] = total_match_logs_df['Tkl'].astype(float)
total_match_logs_df['Int'] = total_match_logs_df['Int'].astype(float)
total_match_logs_df['Blocks'] = total_match_logs_df['Blocks'].astype(float)
total_match_logs_df['xG'] = total_match_logs_df['xG'].astype(float)
total_match_logs_df['npxG'] = total_match_logs_df['npxG'].astype(float)
total_match_logs_df['xA'] = total_match_logs_df['xA'].astype(float)
total_match_logs_df['SCA'] = total_match_logs_df['SCA'].astype(float)
total_match_logs_df['GCA'] = total_match_logs_df['GCA'].astype(float)
total_match_logs_df['Cmp'] = total_match_logs_df['Cmp'].astype(float)
total_match_logs_df['Att'] = total_match_logs_df['Att'].astype(float)
total_match_logs_df['Prog'] = total_match_logs_df['Prog'].astype(float)
total_match_logs_df['Carries'] = total_match_logs_df['Carries'].astype(float)
total_match_logs_df['Prog.1'] = total_match_logs_df['Prog.1'].astype(float)
total_match_logs_df['Succ'] = total_match_logs_df['Succ'].astype(float)

In [39]:
total_match_logs_df_05 = total_match_logs_df.drop_duplicates()

In [40]:
total_match_logs_df_05[(total_match_logs_df_05['year'] == 2006) & (total_match_logs_df_05['FBRefID'] == '71672fa0') & (total_match_logs_df_05['week'] == 41)].sort_values('Date')
# total_match_logs_df_05[(total_match_logs_df_05['FBRefID'] == '71672fa0')].sort_values('Date')

,Date,Day,Comp,Round,Venue,Result,Squad,Opponent,Start,Pos,Min,Gls,Ast,PK,PKatt,Sh,SoT,CrdY,CrdR,Touches,Press,Tkl,Int,Blocks,xG,npxG,xA,SCA,GCA,Cmp,Att,Cmp%,Prog,Carries,Prog.1,Succ,Att.1,name,FBRefID,Fls,Fld,Off,Crs,TklW,OG,PKwon,PKcon,week,year,Won,Loss,Draw,Games_Start
226444,2006-10-11,Wed,Euro Qualifying,Group stage,Away,D 1–1,cz Czech Republic,ie Republic of Ireland,Y,NaN,90.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Petr-Cech,71672fa0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,41,2006,0.0,0.0,1.0,1.0
226445,2006-10-14,Sat,Premier League,Matchweek 8,Away,W 1–0,Chelsea,Reading,Y,NaN,4.0,0.0,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Petr-Cech,71672fa0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,41,2006,1.0,0.0,0.0,1.0


In [41]:
# Eliminated lowercase component of 'Squad' column

def no_lowercase(s):
    lst = []
    for i in s.split(' '):
        if i.islower():
            continue
        else:
            lst.append(i)
    
    return ' '.join(lst)
        # no_lc = ' '.join([word for word in i.split(' ') if not word.islower()])

total_match_logs_df_05['Squad'] = total_match_logs_df_05['Squad'].apply(no_lowercase)


<ipython-input-41-4d58e83e1db8>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_match_logs_df_05['Squad'] = total_match_logs_df_05['Squad'].apply(no_lowercase)


In [42]:
# Sanity Checks

# total_match_logs_df_05[(total_match_logs_df_05['year'] == 2006) & (total_match_logs_df_05['FBRefID'] == '71672fa0')].sort_values('Date')
# dict(total_match_logs_df_05['Squad'].value_counts()[1000:-1])

total_match_logs_df_05.columns

Index(['Date', 'Day', 'Comp', 'Round', 'Venue', 'Result', 'Squad', 'Opponent',
       'Start', 'Pos', 'Min', 'Gls', 'Ast', 'PK', 'PKatt', 'Sh', 'SoT', 'CrdY',
       'CrdR', 'Touches', 'Press', 'Tkl', 'Int', 'Blocks', 'xG', 'npxG', 'xA',
       'SCA', 'GCA', 'Cmp', 'Att', 'Cmp%', 'Prog', 'Carries', 'Prog.1', 'Succ',
       'Att.1', 'name', 'FBRefID', 'Fls', 'Fld', 'Off', 'Crs', 'TklW', 'OG',
       'PKwon', 'PKcon', 'week', 'year', 'Won', 'Loss', 'Draw', 'Games_Start'],
      dtype='object')

In [43]:
# Choosing wanted columns

match_log_columns = ['Date', 'Min', 'Gls', 'Ast', 'PK', 'PKatt', 'Sh', 'SoT', 'CrdY',
 'CrdR', 'Touches', 'Press', 'Tkl', 'Int', 'Blocks', 'xG', 'npxG', 'xA',
 'SCA', 'GCA', 'Cmp', 'Att', 'Prog', 'Carries', 'Prog.1', 'Succ',
 'Att.1', 'name', 'FBRefID', 'Fls', 'Fld', 'Off', 'Crs', 'TklW', 'OG',
 'PKwon', 'PKcon', 'week', 'year', 'Won', 'Loss', 'Draw']

total_match_logs_df_2 = total_match_logs_df_05[match_log_columns]

In [44]:
# Sanity check of total_match_logs_df_2

# total_match_logs_df_2[(total_match_logs_df_2['year'] == 2006) & (total_match_logs_df_2['FBRefID'] == '71672fa0')].sort_values('week')

In [45]:
# Grouping total_match_logs_df_2 by name, FBRefID, week and year

total_match_logs_df_3 = total_match_logs_df_2.groupby(by=['name', 'FBRefID','week', 'year', 'Date']).sum().reset_index()

In [46]:
total_match_logs_df_3.head()

,name,FBRefID,week,year,Date,Min,Gls,Ast,PK,PKatt,Sh,SoT,CrdY,CrdR,Touches,Press,Tkl,Int,Blocks,xG,npxG,xA,SCA,GCA,Cmp,Att,Prog,Carries,Prog.1,Succ,Att.1,Fls,Fld,Off,Crs,TklW,OG,PKwon,PKcon,Won,Loss,Draw
0,Aaron-Connolly,27c01749,1,2020,2020-01-01,45.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,12.0,7.0,0.0,1.0,1.0,0.2,0.2,0.0,1.0,0.0,4.0,5.0,1.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Aaron-Connolly,27c01749,1,2020,2020-01-04,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,Aaron-Connolly,27c01749,2,2018,2018-01-08,80.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,Aaron-Connolly,27c01749,2,2019,2019-01-07,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,Aaron-Connolly,27c01749,3,2019,2019-01-14,81.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [47]:
total_match_logs_df_3[(total_match_logs_df_3['FBRefID'] == '09586899') & (total_match_logs_df_3['week'] == 14) & (total_match_logs_df_3['year'] == 2010)]

,name,FBRefID,week,year,Date,Min,Gls,Ast,PK,PKatt,Sh,SoT,CrdY,CrdR,Touches,Press,Tkl,Int,Blocks,xG,npxG,xA,SCA,GCA,Cmp,Att,Prog,Carries,Prog.1,Succ,Att.1,Fls,Fld,Off,Crs,TklW,OG,PKwon,PKcon,Won,Loss,Draw
226804,Arjen-Robben,09586899,14,2010,2010-04-07,75.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
226805,Arjen-Robben,09586899,14,2010,2010-04-10,90.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [48]:
# Sanity checks for total_match_logs_df_3

# total_match_logs_df_3[(total_match_logs_df_3['FBRefID'] == '71672fa0') & (total_match_logs_df_3['year'] == 2006)].sort_values(['year', 'week'])

player_injuries_profile_final.head()

,Season,Injury,from,until,Days,Games missed,TMId,name_x,PlayerFBref,UrlFBref,UrlTmarkt,TmPos,FBRefID,TMID,FBRefId,PlayerName,PlayerFullName,Nationality,Photo,Birth,Height,Weight,Position,Foot,FootAbility,InternationalReputation,PlayerUrl,Twitter,Instagram,Place of birth:,Citizenship:,Height:,Position:,Contract expires:,Former International:,Caps/Goals:,League level:,Joined:,Contract until:,name_y,Date of birth,Last club:,Most games for:,Unknown since:,Date of birth:,Age:,Foot:,Player agent:,Current club:,Agent:,National player:,Name in home country:,Outfitter:,Current international:,Social-Media:,Full name:,On loan from:,Contract there expires:,Contract option:,Retired since:,club:,Without Club since:,2nd club:,country:,Company:,Date of death:,Career break since:,3nd club:,Ban since:,injury_year,injury_week,release_week,current_date,current_year,current_week
0,18/19,Hamstring Injury,2018-09-29,2018-10-27,28 days,5,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,71672fa0,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,39,43,2018-09-30,2018,39
1,18/19,Hamstring Injury,2018-09-29,2018-10-27,28 days,5,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,71672fa0,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,39,43,2018-10-07,2018,40
2,18/19,Hamstring Injury,2018-09-29,2018-10-27,28 days,5,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,71672fa0,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,39,43,2018-10-14,2018,41
3,18/19,Hamstring Injury,2018-09-29,2018-10-27,28 days,5,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,71672fa0,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,39,43,

In [49]:
player_list = ['Season', 'Injury', 'from', 'until', 'Days', 'Games missed', 'TMId', 'TmPos', 'FBRefID', 'TMID', 'PlayerName', 'PlayerFullName', 'Nationality',
 'Photo', 'Birth', 'Height', 'Weight', 'Position', 'Foot', 'FootAbility', 'InternationalReputation', 'PlayerUrl', 'Twitter', 'Instagram',
 'Place of birth:', 'Citizenship:', 'Position:', 'Contract expires:', 'Former International:', 'Caps/Goals:', 'League level:', 'Joined:', 'Contract until:',
 'Last club:', 'Most games for:', 'Unknown since:', 'Date of birth:', 'Foot:', 'Player agent:', 'Current club:', 'National player:', 'Name in home country:', 'Outfitter:',
 'Current international:', 'Social-Media:', 'Full name:', 'On loan from:', 'Contract there expires:', 'Contract option:', 'Retired since:', 'club:', 'Without Club since:', '2nd club:',
 'country:', 'Company:', 'Date of death:', 'Career break since:', '3nd club:', 'Ban since:', 'injury_year', 'injury_week', 'release_week', 'current_date', 'current_year', 'current_week']

# player_injuries_profile_final.head()
player_injuries_profile_final = player_injuries_profile_final[player_list].reset_index()

In [50]:
player_injuries_profile_final[pd.isna(player_injuries_profile_final['current_date'])]

,index,Season,Injury,from,until,Days,Games missed,TMId,TmPos,FBRefID,TMID,PlayerName,PlayerFullName,Nationality,Photo,Birth,Height,Weight,Position,Foot,FootAbility,InternationalReputation,PlayerUrl,Twitter,Instagram,Place of birth:,Citizenship:,Position:,Contract expires:,Former International:,Caps/Goals:,League level:,Joined:,Contract until:,Last club:,Most games for:,Unknown since:,Date of birth:,Foot:,Player agent:,Current club:,National player:,Name in home country:,Outfitter:,Current international:,Social-Media:,Full name:,On loan from:,Contract there expires:,Contract option:,Retired since:,club:,Without Club since:,2nd club:,country:,Company:,Date of death:,Career break since:,3nd club:,Ban since:,injury_year,injury_week,release_week,current_date,current_year,current_week


In [51]:
total_match_logs_df_3['Date'] = pd.to_datetime(total_match_logs_df_3['Date'])

In [52]:
# Merging total_match_logs_df with player_injuries_profile_final
complete_final_df = pd.merge(left=total_match_logs_df_3, right=player_injuries_profile_final, left_on=['week', 'year', 'Date', 'FBRefID'], right_on=['current_week', 'current_year', 'current_date', 'FBRefID'], how='outer')

In [53]:
len(complete_final_df['FBRefID'].unique())

17178

In [54]:
# Cleaning complete_final_df
complete_final_df.loc[complete_final_df['week'].isna(), 'agg_week'] = complete_final_df['current_week']
complete_final_df.loc[complete_final_df['year'].isna(), 'agg_year'] = complete_final_df['current_year']
complete_final_df.loc[complete_final_df['Date'].isna(), 'agg_date'] = complete_final_df['current_date']

complete_final_df.loc[complete_final_df['current_week'].isna(), 'agg_week'] = complete_final_df['week']
complete_final_df.loc[complete_final_df['current_year'].isna(), 'agg_year'] = complete_final_df['year']
complete_final_df.loc[complete_final_df['current_date'].isna(), 'agg_date'] = complete_final_df['Date']

complete_final_df.loc[complete_final_df['agg_week'].isna(), 'agg_week'] = complete_final_df['week']
complete_final_df.loc[complete_final_df['agg_year'].isna(), 'agg_year'] = complete_final_df['year']
complete_final_df.loc[complete_final_df['agg_date'].isna(), 'agg_date'] = complete_final_df['Date']

In [55]:
keep_columns = ['name', 'FBRefID', 'Min', 'Gls', 'Ast', 'PK', 'PKatt', 'Sh', 'SoT', 'CrdY', 'CrdR', 'Touches', 'Press', 'Tkl', 'Int', 'Blocks', 'xG', 'npxG', 'xA', 'SCA', 
                'GCA', 'Cmp', 'Att', 'Prog', 'Carries', 'Prog.1', 'Succ', 'Att.1', 'Fls', 'Fld', 'Off', 'Crs', 'TklW', 'OG', 'PKwon', 'PKcon', 'Won', 'Loss', 'Draw', 'Season',
                'Injury', 'from', 'until', 'Days', 'Games missed', 'TMId', 'TmPos', 'TMID', 'PlayerName', 'PlayerFullName', 'Nationality', 'Photo', 'Birth', 'Height', 'Weight',
                'Position', 'Foot', 'FootAbility', 'InternationalReputation', 'PlayerUrl', 'Twitter', 'Instagram', 'Place of birth:', 'Citizenship:', 'Position:', 'Contract expires:',
                'Former International:', 'Caps/Goals:', 'League level:', 'Joined:', 'Contract until:', 'Last club:', 'Most games for:', 'Unknown since:','Foot:',
                'Player agent:', 'Current club:', 'National player:', 'Name in home country:', 'Full name:', 'On loan from:', 'Retired since:', 'club:', 'Date of death:', 'Career break since:',
                'Ban since:', 'injury_year', 'injury_week', 'release_week', 'current_date', 'current_year', 'current_week', 'agg_week', 'agg_year', 'agg_date', 'was_match']

In [56]:
# Creating variable 'was_match' to know which rows are matches (real games) and which rows are not

complete_final_df.loc[complete_final_df['Min'].isnull(), 'was_match'] = 0
complete_final_df.loc[complete_final_df['Min'].isnull() == False, 'was_match'] = 1

In [57]:
complete_final_df_2 = complete_final_df[keep_columns].fillna(0).groupby(by=['name', 'FBRefID','agg_week', 'agg_year', 'Injury', 'injury_week', 'injury_year', 'Retired since:', 'Position:', 'PlayerName', 'TMID']).sum().reset_index()

In [58]:
complete_final_df_2.head()

,name,FBRefID,agg_week,agg_year,Injury,injury_week,injury_year,Retired since:,Position:,PlayerName,TMID,Min,Gls,Ast,PK,PKatt,Sh,SoT,CrdY,CrdR,Touches,Press,Tkl,Int,Blocks,xG,npxG,xA,SCA,GCA,Cmp,Att,Prog,Carries,Prog.1,Succ,Att.1,Fls,Fld,Off,Crs,TklW,OG,PKwon,PKcon,Won,Loss,Draw,Height,Weight,FootAbility,Ban since:,release_week,current_year,current_week,was_match
0,0,0000acda,1.0,2021.0,Calf Injury,36.0,2020.0,0,midfield - Central Midfield,Marco Benassi,201040,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,184.0,80.0,77.0,0,21.0,2021.0,1.0,0.0
1,0,0000acda,2.0,2021.0,Calf Injury,36.0,2020.0,0,midfield - Central Midfield,Marco Benassi,201040,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,184.0,80.0,77.0,0,21.0,2021.0,2.0,0.0
2,0,0000acda,3.0,2021.0,Calf Injury,36.0,2020.0,0,midfield - Central Midfield,Marco Benassi,201040,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,184.0,80.0,77.0,0,21.0,2021.0,3.0,0.0
3,0,0000acda,4.0,2021.0,Calf Injury,36.0,2020.0,0,midfield - Central Midfield,Marco Benassi,201040,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,184.0,80.0,77.0,0,21.0,2021.0,4.0,0.0
4,0,0000acda,5.0,2021.0,Calf Injury,36.0,2020.0,0,midfield - Central Midfield,Marco Benassi,201040,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,184.0,80.0,77.0,0,21.0,2021.0,5.0,0.0


In [59]:
# Populating Empty 'week' and 'year'
# df.loc[df['foo'].isnull(),'foo'] = df['bar']

# complete_final_df.loc[complete_final_df['week'].isnull(), 'week'] = complete_final_df['agg_week']
# complete_final_df.loc[complete_final_df['year'].isnull(), 'year'] = complete_final_df['agg_year']
# complete_final_df.loc[complete_final_df['Date'].isnull(), 'Date'] = complete_final_df['agg_date']

In [60]:
len(complete_final_df_2['FBRefID'].unique())

17178

In [61]:
complete_final_df_2 = complete_final_df_2[complete_final_df_2['FBRefID'].isin(list(fbref_to_tm_df['FBRefID'].unique()))]

In [62]:
# Sorting complete_final_df
pd.set_option("display.max_columns", None)
complete_final_df_2.sort_values(['FBRefID', 'agg_year', 'agg_week'], inplace=True)

In [63]:
# Sanity Checks complete_final_df
pd.set_option("display.max_rows", 5000)
# complete_final_df[(complete_final_df['FBRefID'] == '71672fa0') & (complete_final_df['agg_year'] == 2006)]
# complete_final_df[(complete_final_df['FBRefID'] == '71672fa0')]

In [64]:
complete_final_df_2.head()

,name,FBRefID,agg_week,agg_year,Injury,injury_week,injury_year,Retired since:,Position:,PlayerName,TMID,Min,Gls,Ast,PK,PKatt,Sh,SoT,CrdY,CrdR,Touches,Press,Tkl,Int,Blocks,xG,npxG,xA,SCA,GCA,Cmp,Att,Prog,Carries,Prog.1,Succ,Att.1,Fls,Fld,Off,Crs,TklW,OG,PKwon,PKcon,Won,Loss,Draw,Height,Weight,FootAbility,Ban since:,release_week,current_year,current_week,was_match
1439339,Marco-Benassi,0000acda,31.0,2012.0,0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,1.0
1439342,Marco-Benassi,0000acda,32.0,2012.0,0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,1.0
1439350,Marco-Benassi,0000acda,34.0,2012.0,0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,1.0
1439359,Marco-Benassi,0000acda,35.0,2012.0,0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.0,0.0,0.0,1.0
1439392,Marco-Benassi,0000acda,40.0,2012.0,0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,1.0


In [65]:
# Sravan's path
# player_weight_foot_df = pd.read_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/Dataframes/player_profile_weight.csv')

# Renzo's path
player_weight_foot_df = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_profile_weight.csv')

player_weight_foot_df.head()

,FBRefId,Height,Weight,Foot,Birth
0,3bb7b8b4,188.0,89.0,LEFT,"August 17, 1993"
1,31c69ef1,186.0,82.0,RIGHT,"May 14, 1997"
2,6434f10d,190.0,78.0,RIGHT,"June 22, 1996"
3,b400bde0,170.0,70.0,RIGHT,"December 8, 1994"
4,bd6351cd,182.0,66.0,RIGHT,"May 27, 1994"


In [66]:
complete_final_df_2.drop(columns=['Height', 'Weight'], axis=1, inplace=True)

complete_final_df_3 = pd.merge(complete_final_df_2, player_weight_foot_df, left_on='FBRefID', right_on='FBRefId', how='inner')

len(complete_final_df_3['FBRefID'].unique())

5370

In [67]:
# Change Birth field to Datetime
complete_final_df_3['Birth'] = pd.to_datetime(complete_final_df_3['Birth'])

In [68]:
complete_final_df_3.head()

,name,FBRefID,agg_week,agg_year,Injury,injury_week,injury_year,Retired since:,Position:,PlayerName,TMID,Min,Gls,Ast,PK,PKatt,Sh,SoT,CrdY,CrdR,Touches,Press,Tkl,Int,Blocks,xG,npxG,xA,SCA,GCA,Cmp,Att,Prog,Carries,Prog.1,Succ,Att.1,Fls,Fld,Off,Crs,TklW,OG,PKwon,PKcon,Won,Loss,Draw,FootAbility,Ban since:,release_week,current_year,current_week,was_match,FBRefId,Height,Weight,Foot,Birth
0,Marco-Benassi,0000acda,31.0,2012.0,0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.0,0.0,0.0,1.0,0000acda,184.0,80.0,RIGHT,1994-09-08
1,Marco-Benassi,0000acda,32.0,2012.0,0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0.0,0.0,0.0,1.0,0000acda,184.0,80.0,RIGHT,1994-09-08
2,Marco-Benassi,0000acda,34.0,2012.0,0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.0,0.0,0.0,1.0,0000acda,184.0,80.0,RIGHT,1994-09-08
3,Marco-Benassi,0000acda,35.0,2012.0,0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0.0,0.0,0.0,1.0,0000acda,184.0,80.0,RIGHT,1994-09-08
4,Marco-Benassi,0000acda,40.0,2012.0,0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.0,0.0,0.0,1.0,0000acda,184.0,80.0,RIGHT,1994-09-08


In [69]:
complete_final_df_3["first_week"] = complete_final_df_3.groupby("FBRefID")["agg_week"].transform("first")
complete_final_df_3["first_year"] = complete_final_df_3.groupby("FBRefID")["agg_year"].transform("first")

complete_final_df_3['week_diff'] = complete_final_df_3["agg_week"] - complete_final_df_3["first_week"]
complete_final_df_3['year_diff'] = complete_final_df_3["agg_year"] - complete_final_df_3["first_year"]
complete_final_df_3['cum_week'] = complete_final_df_3["week_diff"] + complete_final_df_3["year_diff"]*52

# range_cum_week = list(range(int(min(complete_final_df_3['cum_week'])), int(max(complete_final_df_3['cum_week'])+1)))
# fbref_ids_list = list(np.repeat(sorted(complete_final_df_3['FBRefID'].unique()), len(range_cum_week)))
# cum_week_list = range_cum_week * len(complete_final_df_3['FBRefID'].unique())

# cum_df = pd.DataFrame(columns=['week', 'FBRefID'])
# cum_df['week'] = cum_week_list
# cum_df['FBRefID'] = fbref_ids_list

# cum_df.head()

In [70]:
range_cum_year = list(range(int(min(complete_final_df_3['agg_year'])), int(max(complete_final_df_3['agg_year'])+1)))

range_week_year = list(range(1,53))

cum_year_list = range_cum_year * len(range_week_year) * len(complete_final_df_3['FBRefID'].unique())
cum_week_list = range_week_year * len(range_cum_year) * len(complete_final_df_3['FBRefID'].unique())
fbref_ids_list = list(np.repeat(sorted(complete_final_df_3['FBRefID'].unique()), len(cum_week_list)))

# len(cum_year_list), len(cum_week_list), len(fbref_ids_list)

In [ ]:
cum_year_list

Error: Session cannot generate requests

In [ ]:
# cum_df = pd.DataFrame(columns=['week', 'year', 'FBRefID'])
# cum_df['week'] = cum_week_list
# cum_df['year'] = cum_year_list
# cum_df['FBRefID'] = fbref_ids_list

# cum_df.head()

Error: Session cannot generate requests

In [ ]:
complete_final_df_4 = pd.merge(cum_df, complete_final_df_3, left_on=['week', 'FBRefID'], right_on=['cum_week', 'FBRefID'], how='left')
complete_final_df_4.head()

,week,FBRefID,name,agg_week,agg_year,Injury,injury_week,injury_year,Retired since:,Position:,PlayerName,TMID,Min,Gls,Ast,PK,PKatt,Sh,SoT,CrdY,CrdR,Touches,Press,Tkl,Int,Blocks,xG,npxG,xA,SCA,GCA,Cmp,Att,Prog,Carries,Prog.1,Succ,Att.1,Fls,Fld,Off,Crs,TklW,OG,PKwon,PKcon,Won,Loss,Draw,FootAbility,Ban since:,release_week,current_year,current_week,was_match,FBRefId,Height,Weight,Foot,Birth,first_week,first_year,week_diff,year_diff,cum_week
0,0,0000acda,0,46.0,2013.0,Calf Problems,46.0,2013.0,0,midfield - Central Midfield,Marco Benassi,201040,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,77.0,0.0,47.0,2013.0,46.0,0.0,0000acda,184.0,80.0,RIGHT,1994-09-08,46.0,2013.0,0.0,0.0,0.0
1,1,0000acda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
2,2,0000acda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
3,3,0000acda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
4,4,0000acda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN


In [ ]:
complete_final_df_4.loc[pd.isna(complete_final_df_4['cum_week']), 'cum_week'] = complete_final_df_4['week']

complete_final_df_4['name'] = complete_final_df_4['name'].fillna(method='ffill')
# complete_final_df_4['agg_week_1'] = complete_final_df_4['agg_week'].fillna(method='ffill')

# complete_final_df_4['agg_week'] = complete_final_df_4['agg_week'].ffill()
# complete_final_df_4.loc[complete_final_df_4['agg_week'] == complete_final_df_4['agg_week'].shift(1), 'agg_week'] = complete_final_df_4['agg_week'].shift(1) + 1
# complete_final_df_4.loc[complete_final_df_4['agg_week'] > 52, 'agg_week'] = complete_final_df_4['agg_week'] - 52
# complete_final_df_4.loc[pd.isna(complete_final_df_4['cum_week']), 'cum_week'] = complete_final_df_4['cum_week'].shift(1) + 1
# complete_final_df_4.loc[pd.isna(complete_final_df_4['name']), 'name'] = complete_final_df_4['name'].shift(1)

ref_df = complete_final_df_4[['FBRefID', 'first_year', 'first_week']].drop_duplicates()
ref_df.dropna(inplace=True)

complete_final_df_5 = pd.merge(complete_final_df_4.drop(columns=['first_year', 'first_week']), ref_df,left_on=['FBRefID'], right_on=['FBRefID'], how='left')

# complete_final_df_4.loc[(pd.isna(complete_final_df_4['first_week'])), 'first_week'] = complete_final_df_4['first_week'].shift(1)
# complete_final_df_4.loc[(pd.isna(complete_final_df_4['first_year'])), 'first_year'] = complete_final_df_4['first_year'].shift(1)
# complete_final_df_4.loc[(pd.isna(complete_final_df_4['agg_week'])) & (complete_final_df_4['agg_week'].shift(1) + 1 <= 52), 'agg_week'] = complete_final_df_4['agg_week'].shift(1) + 1

In [ ]:
# def sum_from_above(row, df):
#     if row['agg_week'] == row['agg_week_1']:
#         return row['agg_week'] + 1
#     else:
#         return row['agg_week']

nul = complete_final_df_5['agg_week'].isnull()
complete_final_df_5['agg_week_check'] = nul.groupby((nul.diff() == 1).cumsum()).cumsum() + complete_final_df_5['agg_week'].ffill()
complete_final_df_5.loc[complete_final_df_5['agg_week_check'] > 1092, 'agg_week_check'] = complete_final_df_5['agg_week_check'] - 1092
complete_final_df_5.loc[complete_final_df_5['agg_week_check'] > 1040, 'agg_week_check'] = complete_final_df_5['agg_week_check'] - 1040
complete_final_df_5.loc[complete_final_df_5['agg_week_check'] > 988, 'agg_week_check'] = complete_final_df_5['agg_week_check'] - 988
complete_final_df_5.loc[complete_final_df_5['agg_week_check'] > 936, 'agg_week_check'] = complete_final_df_5['agg_week_check'] - 936
complete_final_df_5.loc[complete_final_df_5['agg_week_check'] > 884, 'agg_week_check'] = complete_final_df_5['agg_week_check'] - 884
complete_final_df_5.loc[complete_final_df_5['agg_week_check'] > 832, 'agg_week_check'] = complete_final_df_5['agg_week_check'] - 832
complete_final_df_5.loc[complete_final_df_5['agg_week_check'] > 780, 'agg_week_check'] = complete_final_df_5['agg_week_check'] - 780
complete_final_df_5.loc[complete_final_df_5['agg_week_check'] > 728, 'agg_week_check'] = complete_final_df_5['agg_week_check'] - 728
complete_final_df_5.loc[complete_final_df_5['agg_week_check'] > 676, 'agg_week_check'] = complete_final_df_5['agg_week_check'] - 676
complete_final_df_5.loc[complete_final_df_5['agg_week_check'] > 624, 'agg_week_check'] = complete_final_df_5['agg_week_check'] - 624
complete_final_df_5.loc[complete_final_df_5['agg_week_check'] > 572, 'agg_week_check'] = complete_final_df_5['agg_week_check'] - 572
complete_final_df_5.loc[complete_final_df_5['agg_week_check'] > 520, 'agg_week_check'] = complete_final_df_5['agg_week_check'] - 520
complete_final_df_5.loc[complete_final_df_5['agg_week_check'] > 468, 'agg_week_check'] = complete_final_df_5['agg_week_check'] - 468
complete_final_df_5.loc[complete_final_df_5['agg_week_check'] > 416, 'agg_week_check'] = complete_final_df_5['agg_week_check'] - 416
complete_final_df_5.loc[complete_final_df_5['agg_week_check'] > 364, 'agg_week_check'] = complete_final_df_5['agg_week_check'] - 364
complete_final_df_5.loc[complete_final_df_5['agg_week_check'] > 312, 'agg_week_check'] = complete_final_df_5['agg_week_check'] - 312
complete_final_df_5.loc[complete_final_df_5['agg_week_check'] > 260, 'agg_week_check'] = complete_final_df_5['agg_week_check'] - 260
complete_final_df_5.loc[complete_final_df_5['agg_week_check'] > 208, 'agg_week_check'] = complete_final_df_5['agg_week_check'] - 208
complete_final_df_5.loc[complete_final_df_5['agg_week_check'] > 156, 'agg_week_check'] = complete_final_df_5['agg_week_check'] - 156
complete_final_df_5.loc[complete_final_df_5['agg_week_check'] > 104, 'agg_week_check'] = complete_final_df_5['agg_week_check'] - 104
complete_final_df_5.loc[complete_final_df_5['agg_week_check'] > 52, 'agg_week_check'] = complete_final_df_5['agg_week_check'] - 52

complete_final_df_5['agg_week'] = complete_final_df_5['agg_week_check']
complete_final_df_5['agg_week'].drop(columns=['agg_week_check'], axis=1, inplace=True)
    
# complete_final_df_5['agg_week_check'] = complete_final_df_5.apply(lambda x: sum_from_above(x), axis=1)

# complete_final_df_5['agg_week_1'] = complete_final_df_5['agg_week'].shift(1)

# complete_final_df_5.loc[complete_final_df_5['agg_week'] == complete_final_df_5['agg_week_1'], 'agg_week'] = complete_final_df_5.groupby('FBRefID')['agg_week'].apply(lambda x: x.shift(1) + 1)

In [ ]:
# nul = complete_final_df_5.loc[complete_final_df_5['agg_week'] == 1]['agg_year'].isnull()
# complete_final_df_5.loc[complete_final_df_5['agg_week'] == 1]['agg_year'] = nul.groupby((nul.diff() == 1).cumsum()).cumsum() + complete_final_df_5.loc[complete_final_df_5['agg_week'] == 1]['agg_year'].ffill()

# complete_final_df_5.loc[complete_final_df_5['agg_week'] == 1]

# complete_final_df_5['agg_year_1'] = complete_final_df_5['agg_year'].shift(1)
# complete_final_df_5['agg_week_1'] = complete_final_df_5['agg_week'].shift(1)

# complete_final_df_5.loc[(pd.isna(complete_final_df_5['agg_year'])) & (complete_final_df_5['agg_week_1'] == 52), 'agg_year_check'] = complete_final_df_5['agg_year_1'] + 1
# complete_final_df_5.loc[(pd.isna(complete_final_df_5['agg_year'])) & (complete_final_df_5['agg_week_1'] < 52), 'agg_year_check'] = complete_final_df_5['agg_year_1']

# complete_final_df_5['agg_year'].fillna(method='ffill', inplace=True)

# complete_final_df_5.loc[(complete_final_df_5['agg_week'] == 1) & (pd.isna(complete_final_df_5['agg_year'])), 'agg_year'] = complete_final_df_5['agg_year'].shift(1) + 1
# complete_final_df_5.loc[(complete_final_df_5['agg_week'] > 1) & (complete_final_df_5['agg_year'] != complete_final_df_5['agg_year'].shift(1)), 'agg_year']

In [ ]:
columns_to_zero = ['Min', 'Gls', 'Ast', 'PK', 'PKatt', 'Sh', 'SoT',
'CrdY', 'CrdR', 'Touches', 'Press', 'Tkl', 'Int', 'Blocks', 'xG',
'npxG', 'xA', 'SCA', 'GCA', 'Cmp', 'Att', 'Prog', 'Carries', 'Prog.1',
'Succ', 'Att.1', 'Fls', 'Fld', 'Off', 'Crs', 'TklW', 'OG', 'PKwon',
'PKcon', 'Won', 'Loss', 'Draw']

for i in columns_to_zero:
    complete_final_df_5[i].fillna(0)
    

In [ ]:
# complete_final_df_5[complete_final_df_5['FBRefID']=='71672fa0'].head(10)


complete_final_df_5[complete_final_df_5['agg_year']==1991]

,week,FBRefID,name,agg_week,agg_year,Injury,injury_week,injury_year,Retired since:,Position:,PlayerName,TMID,Min,Gls,Ast,PK,PKatt,Sh,SoT,CrdY,CrdR,Touches,Press,Tkl,Int,Blocks,xG,npxG,xA,SCA,GCA,Cmp,Att,Prog,Carries,Prog.1,Succ,Att.1,Fls,Fld,Off,Crs,TklW,OG,PKwon,PKcon,Won,Loss,Draw,FootAbility,Ban since:,release_week,current_year,current_week,was_match,FBRefId,Height,Weight,Foot,Birth,week_diff,year_diff,cum_week,first_year,first_week,agg_week_check
5690085,0,98ee8a67,Edwin-van-der-Sar,38.0,1991.0,0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,98ee8a67,197.0,85.0,RIGHT,1970-10-29,0.0,0.0,0.0,1991.0,38.0,38.0
5690087,2,98ee8a67,Edwin-van-der-Sar,40.0,1991.0,0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,98ee8a67,197.0,85.0,RIGHT,1970-10-29,2.0,0.0,2.0,1991.0,38.0,40.0
5690090,5,98ee8a67,Edwin-van-der-Sar,43.0,1991.0,0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,98ee8a67,197.0,85.0,RIGHT,1970-10-29,5.0,0.0,5.0,1991.0,38.0,43.0
5690092,7,98ee8a67,Edwin-van-der-Sar,45.0,1991.0,0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,98ee8a67,197.0,85.0,RIGHT,1970-10-29,7.0,0.0,7.0,1991.0,38.0,45.0
5690095,10,98ee8a67,Edwin-van-der-Sar,48.0,1991.0,0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,98ee8a67,197.0,85.0,RIGHT,1970-10-29,10.0,0.0,10.0,1991.0,38.0,48.0
5690097,12,98ee8a67,Edwin-van-der-Sar,50.0,1991.0,0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,98ee8a67,197.0,85.0,RIGHT,1970-10-29,12.0,0.0,12.0,1991.0,38.0,50.0
7704051,0,d1ca1129,Jens-Lehmann,31.0,1991.0,0,0.0,0.0,0,0,0,0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,d1ca1129,190.0,87.0,NaN,1969-11-10,0.0,0.0,0.0,1991.0,31.0,31.0
7704052,1,d1ca1129,Jens-Lehmann,32.0,1991.0,0,0.0,0.0,0,0,0,0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,d1ca1129,190.0,87.0,NaN,1969-11-10,1.0,0.0,1.0,1991.0,31.0,32.0
7704053,2,d1ca1129,Jens-Lehmann,33.0,1991.0,0,0.0,0.0,0,0,0,0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,d1ca1129,190.0,87.0,NaN,1969-11-10,2.0,0.0,2.0,1991.0,31.0,33.0
7704054,3,d1ca1129,Jens-Lehmann,34.0,1991.0,0,0.0,0.0,0,0,0,0,180.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,d1ca1129,190.0,87.0,NaN,1969-11-10,3.0,0.0,3.0,1991.0,31.0,34.0


In [99]:
# Saving complete_final_df to Google Drive

# Renzo's path
complete_final_df_3.to_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/complete_final_df_3.csv', index=False)

# Sravan's path
# complete_final_df_3.to_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/Dataframes/complete_final_df_3.csv', index=False)